In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import os
import pandas as pd
import cv2
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import time

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



res_labels ={ 0 :'airplane', 1 :'automobile', 2 :'bird', 3 :'cat', 4 :'deer', 5 :'dog', 6 :'frog', 7 :'horse', 8 :'ship', 9 :'truck'}

d1={}
for i in res_labels:
    d1[res_labels[i]] = i
print(d1)

train_dir = 'train'
filenames = os.listdir(train_dir)
filenames.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))


df = pd.read_csv('trainLabels.csv')
labels = [d1[label] for label in df['label'].to_list()]

dataset = list(zip(filenames, labels))

def collate_fn(batch):
    images = []
    labels = []
    for filename, label in batch:
        
        img_path = os.path.join(train_dir, filename)
        im = cv2.imread(img_path)

        if im is None:
            raise RuntimeError(f"Failed to load image: {img_path}")  

        im = cv2.resize(im, (128, 128))
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        im = torch.tensor(im, dtype=torch.float32).permute(2, 0, 1) / 255.0
        images.append(im)
        labels.append(label)

    return torch.stack(images), torch.tensor(labels, dtype=torch.long) 


batch_size = 128 
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,  
    collate_fn=collate_fn,
    persistent_workers=False
)



In [ ]:
model = nn.Sequential(
    nn.Conv2d(3,16,5),# 3*128*128 => 16*124*124
    nn.ReLU(),
    nn.BatchNorm2d(16),
    nn.MaxPool2d(2,2),# 16*124*124 => 16*62*62

    nn.Conv2d(16,32,3),# 16*62*62 => 32*60*60
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.MaxPool2d(2,2),# 32*60*60 => 32*30*30

    nn.Conv2d(32,32,3),# 32*30*30 => 32*28*28
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.MaxPool2d(2,2),# 32*28*28 => 32*14*14

    nn.Conv2d(32,32,5),# 32*14*14 => 32*10*10
    nn.ReLU(),
    nn.BatchNorm2d(32),

    nn.Flatten(),

    nn.Linear(32*10*10,128),
    nn.ReLU(),
    nn.BatchNorm1d(128),

    nn.Linear(128,64),
    nn.ReLU(),
    nn.BatchNorm1d(64),

    nn.Linear(64,10)
)

# Move model to CUDA device
model.to(device)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

epoch=10
start_train_time = time.time() 
for i in range(epoch):
    epoch_loss = 0
    correct = 0
    epoch_start_time = time.time()
    for batch_images , batch_labels in dataloader:
        # Move data to CUDA device
        batch_images = batch_images.to(device)
        batch_labels = batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(batch_images)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()


        epoch_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_labels).sum().item()

    accuracy = 100 * correct / 50000
    epoch_duration = time.time() - epoch_start_time # Calculate epoch duration
    print(f"Epoch {i+1}, Accuracy: {accuracy:.2f}%, Epoch Time: {epoch_duration:.2f}s") # Print epoch time

train_duration = time.time() - start_train_time # Calculate total train time
print(f"Total Training Time: {train_duration:.2f}s")

model_path = 'cifar_model.pth'
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")


In [ ]:

test_dir = 'test'  #
test_filenames = os.listdir(test_dir)
test_filenames.sort(key=lambda f: int(''.join(filter(str.isdigit, f)))) 


test_dataset = test_filenames


# Define collate function for test dataset (no labels needed)
def test_collate_fn(batch_filenames):
    images = []
    image_ids = [] # To extract ID from filename
    for filename in batch_filenames:
        # Load and preprocess image
        img_path = os.path.join(test_dir, filename)
        im = cv2.imread(img_path)

        if im is None:
            raise RuntimeError(f"Failed to load image: {img_path}")

        im = cv2.resize(im, (128, 128))
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        im = torch.tensor(im, dtype=torch.float32).permute(2, 0, 1) / 255.0
        images.append(im)
        image_ids.append(''.join(filter(str.isdigit, filename))) # Extract ID from filename

    return torch.stack(images), image_ids


# Create DataLoader for test dataset
test_batch_size = 256 # Increased test batch size
test_dataloader = DataLoader(
    test_dataset,
    batch_size=test_batch_size,
    shuffle=False, # No need to shuffle test data
    num_workers=0, # Increased num_workers for test dataloader
    collate_fn=test_collate_fn,
    persistent_workers=False
)

In [ ]:
# --- Load Trained Model Weights ---
# --- Model Definition (MUST be the same as your trained model) ---

model = nn.Sequential( # Define your model architecture - MUST MATCH TRAINING MODEL
    nn.Conv2d(3,16,5),# 3*128*128 => 16*124*124
    nn.ReLU(),
    nn.BatchNorm2d(16),
    nn.MaxPool2d(2,2),# 16*124*124 => 16*62*62

    nn.Conv2d(16,32,3),# 16*62*62 => 32*60*60
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.MaxPool2d(2,2),# 32*60*60 => 32*30*30

    nn.Conv2d(32,32,3),# 32*30*30 => 32*28*28
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.MaxPool2d(2,2),# 32*28*28 => 32*14*14

    nn.Conv2d(32,32,5),# 32*14*14 => 32*10*10
    nn.ReLU(),
    nn.BatchNorm2d(32),

    nn.Flatten(),

    nn.Linear(32*10*10,128),
    nn.ReLU(),
    nn.BatchNorm1d(128),

    nn.Linear(128,64),
    nn.ReLU(),
    nn.BatchNorm1d(64),

    nn.Linear(64,10)
)
model.to(device) # Ensure model is on CUDA
# **IMPORTANT:** Replace 'cifar_model.pth' with the actual path if you saved it with a different name or location.
model.load_state_dict(torch.load('cifar_model.pth'))
model.eval() # Set model to evaluation mode
print("Trained model weights loaded from cifar_model.pth")

In [ ]:


output_lines = ["id,label"] # Header for CSV format
count = 0
start_test_time = time.time() # Start timer for testing
with torch.no_grad(): # Disable gradient calculation for inference
    count+=1
    if count%100 ==0:
        print(count)
    for batch_images, image_ids in test_dataloader:
        batch_images = batch_images.to(device) # Move test images to CUDA

        outputs = model(batch_images) # Get model predictions
        _, predicted_labels = torch.max(outputs, 1) # Get predicted class indices

        predicted_labels_cpu = predicted_labels.cpu().numpy() # Move predictions to CPU for processing

        for i in range(len(image_ids)):
            image_id = image_ids[i]
            predicted_label_index = predicted_labels_cpu[i]
            predicted_label_word = res_labels[predicted_label_index]
            output_line = f"{image_id},{predicted_label_word}"
            output_lines.append(output_line)


test_duration = time.time() - start_test_time 
print(f"Total Testing Time: {test_duration:.2f}s")

with open('test_labels.csv', 'w') as f:
    for line in output_lines:
        f.write(line + '\n')

print("Test labels (id,label) are generated and saved to test_labels.csv")